In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import matplotlib.pyplot as plt
import scipy.stats as stats
from google.colab import drive
import seaborn as sns
from scipy.stats import mannwhitneyu
import itertools


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree



Subida de archivo al entorno- Se encuentra en la carpeta SABER PRO en documentos
El archivo ya está segmentado según el grupo de referencia y el carácter de la institución

In [ ]:
#data= pd.read_csv ("https://github.com/yamolina/SABER-PRO/raw/main/CIENCIAS_ECONOMICAS_.csv")
data= pd.read_excel ("https://github.com/yamolina/SABER-PRO/raw/main/CIENCIAS_ECONOMICAS_OK.xlsx")

In [ ]:
#data = pd.read_excel("CIENCIAS_ECONOMICAS_OK.xlsx", engine='openpyxl')

In [ ]:
# Obtener la lista de todas las columnas
columnas = data.columns.tolist()

# Imprimir la lista de columnas
print(columnas)

#***Descripción de los Datos***

## descripción

In [ ]:
data['ESTU_NIVEL_PRGM_ACADEMICO'].value_counts()

In [ ]:
data['GRUPOREFERENCIA'].value_counts()

En total hay 22165 registros con 109 variables

In [ ]:
data['INST_CARACTER_ACADEMICO'].value_counts()

In [ ]:
data ['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

In [ ]:
data = data[data['ESTU_VALORMATRICULAUNIVERSIDAD']!= "No pagó matrícula"] #selección de registros de estudiantes que han pagado la matrícula universitaria y excluyendo los registros de estudiantes que no la han pagado.

In [ ]:
data.dropna(subset=['ESTU_VALORMATRICULAUNIVERSIDAD'], inplace=True)

In [ ]:
data ['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts()

In [ ]:
data ['ESTU_METODO_PRGM'].value_counts()

In [ ]:
# Realizas la agrupación por las categorías deseadas
grouped = data.groupby(['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_METODO_PRGM', 'INST_CARACTER_ACADEMICO'])

# Calculas el promedio de 'PUNT_GLOBAL'
mean_scores = grouped['PUNT_GLOBAL'].mean().reset_index(name='MEAN_PUNT_GLOBAL')

# Calculas el conteo para cada grupo usando la misma agrupación
counts = grouped['PUNT_GLOBAL'].count().reset_index(name='COUNT')

# Combina los resultados del promedio con los conteos
result = pd.merge(mean_scores, counts, on=['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_METODO_PRGM', 'INST_CARACTER_ACADEMICO'])

result

In [ ]:
data = data[~((data['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Más de 7 millones') & (data['ESTU_METODO_PRGM'] != 'PRESENCIAL'))]

In [ ]:
data = data[~((data['ESTU_VALORMATRICULAUNIVERSIDAD'] == 'Más de 7 millones') & (data['INST_CARACTER_ACADEMICO'] == 'INSTITUCIÓN TECNOLÓGICA'))]


## exploración y test de distribucion

###**Valor Matrícula**

In [ ]:
h1 = data.groupby(['ESTU_VALORMATRICULAUNIVERSIDAD'])['PUNT_GLOBAL'].mean().reset_index()
h1 = h1.sort_values(by=['PUNT_GLOBAL'], ascending=True)
print(h1)

In [ ]:
#test de kruskal

l = data.ESTU_VALORMATRICULAUNIVERSIDAD.value_counts().index
pvalues = []
s_ = []

for k,m in enumerate(l):
  s = data.loc[data["ESTU_VALORMATRICULAUNIVERSIDAD"]== m,"PUNT_GLOBAL"]
  s_.append(s)
  test = stats.normaltest(s)
  pvalues.append(test.pvalue)

kw_test = stats.kruskal(*s_)

In [ ]:
#resultado importante del test (medianas por ser no parametrico)
kw_test

In [ ]:
pvalues

In [ ]:
# Supongamos que 's_' es tu lista de series, donde cada serie corresponde a un grupo diferente.
# El número total de comparaciones es n*(n-1)/2, donde n es el número de grupos.

# Calcula el número total de comparaciones
num_comparisons = len(s_) * (len(s_) - 1) / 2

# Nivel de significancia ajustado
alpha_adjusted = 0.05 / num_comparisons

# Lista para almacenar los resultados
post_hoc_results = []

# Itera sobre cada combinación única de pares de grupos
for group1, group2 in itertools.combinations(range(len(s_)), 2):
    stat, pvalue = mannwhitneyu(s_[group1], s_[group2], alternative='two-sided')
    # Añade los resultados a la lista
    post_hoc_results.append({
        'Grupo1': group1,
        'Grupo2': group2,
        'U-Statistic': stat,
        'P-Value': pvalue,
        'Significativo': pvalue < alpha_adjusted
    })

# Convierte los resultados a un DataFrame para una mejor visualización
post_hoc_df = pd.DataFrame(post_hoc_results)

# Muestra los resultados
print(post_hoc_df)


In [ ]:
vmatricula = {
    "Menos de 500 mil":"menos de 1 millon",
    "Entre 500 mil y menos de 1 millón":"menos de 1 millon",
    "Entre 1 millón y menos de 2.5 millones":"entre 1 y 2.5 millones",
    "Entre 2.5 millones y menos de 4 millones":"entre 2.5 y 4 millones",
    "Entre 4 millones y menos de 5.5 millones": "entre 4 y 5.5 millones",
    "Entre 5.5 millones y menos de 7 millones": "entre 5.5 y 7 millones"

} #Asignación de categorías según el valor pagado en la matrícula

In [ ]:
data['ESTU_VALORMATRICULAUNIVERSIDADB']= data['ESTU_VALORMATRICULAUNIVERSIDAD'].replace(vmatricula) #creación de nueva columna en el DataFrame 'data' que contiene los valores de matrícula universitaria transformados utilizando el diccionario vmatricula.

In [ ]:
h1B = data.groupby(['ESTU_VALORMATRICULAUNIVERSIDADB'])['PUNT_GLOBAL'].mean().reset_index()
#h3.describe()
h1B = h1B.sort_values(by = ['PUNT_GLOBAL'], ascending = True)
print(h1B)

In [ ]:
h1B = data.groupby(['ESTU_VALORMATRICULAUNIVERSIDADB']).mean()
#h3.describe()
h1B = h1B.sort_values(by = ['PUNT_GLOBAL'], ascending = True)

fig, ax = plt.subplots(1,1,figsize=(10,5), dpi = 100)
mat_punt = sns.violinplot(x = data["ESTU_VALORMATRICULAUNIVERSIDADB"], y = data["PUNT_GLOBAL"], ax = ax, order=h1B.index)
labs = ax.set(xlabel='Valor', ylabel='Puntaje')
plt.xticks(rotation = -30, ha = "left")

In [ ]:
data["ESTU_VALORMATRICULAUNIVERSIDADB"].value_counts()

###**MÉTODO DE ESTUDIO**

In [ ]:
h2 = data.groupby(['ESTU_METODO_PRGM'])['PUNT_GLOBAL'].mean().reset_index()
h2 = h2.sort_values(by=['PUNT_GLOBAL'], ascending=True)
print(h2)

In [ ]:
h2A = data.groupby(['ESTU_METODO_PRGM']).describe()['PUNT_GLOBAL'].reset_index()
#h2A = h7.sort_values(by = ['PUNT_GLOBAL'], ascending = True)
h2A

In [ ]:
#test de kruskal

l = data.ESTU_METODO_PRGM.value_counts().index
pvalues = []
s_ = []

for k,m in enumerate(l):
  s = data.loc[data["ESTU_METODO_PRGM"]== m,"PUNT_GLOBAL"]
  s_.append(s)
  test = stats.normaltest(s)
  pvalues.append(test.pvalue)

kw_test = stats.kruskal(*s_)
#resultado importante del test (medianas por ser no parametrico)
kw_test

In [ ]:
pvalues

In [ ]:
# Supongamos que 's_' es tu lista de series, donde cada serie corresponde a un grupo diferente.
# El número total de comparaciones es n*(n-1)/2, donde n es el número de grupos.

# Calcula el número total de comparaciones
num_comparisons = len(s_) * (len(s_) - 1) / 2

# Nivel de significancia ajustado
alpha_adjusted = 0.05 / num_comparisons

# Lista para almacenar los resultados
post_hoc_results = []

# Itera sobre cada combinación única de pares de grupos
for group1, group2 in itertools.combinations(range(len(s_)), 2):
    stat, pvalue = mannwhitneyu(s_[group1], s_[group2], alternative='two-sided')
    # Añade los resultados a la lista
    post_hoc_results.append({
        'Grupo1': group1,
        'Grupo2': group2,
        'U-Statistic': stat,
        'P-Value': pvalue,
        'Significativo': pvalue < alpha_adjusted
    })

# Convierte los resultados a un DataFrame para una mejor visualización
post_hoc_df = pd.DataFrame(post_hoc_results)

# Muestra los resultados
print(post_hoc_df)

In [ ]:
h7 = data.groupby(['ESTU_METODO_PRGM']).describe()
#h7 = h7.sort_values(by = ['PUNT_GLOBAL'], ascending = True)
h7
fig, ax = plt.subplots(1,1,figsize=(10,5), dpi = 100)
mat_punt = sns.violinplot(x = data["ESTU_METODO_PRGM"], y = data["PUNT_GLOBAL"], ax = ax, order=h7.index)
labs = ax.set(xlabel='Condición', ylabel='Puntaje')
plt.xticks(rotation = -30, ha = "left")

###**Carácter Académico**

In [ ]:
data['INST_CARACTER_ACADEMICO'].unique()

In [ ]:
h2 = data.groupby(['INST_CARACTER_ACADEMICO'])['PUNT_GLOBAL'].mean().reset_index()
h2 = h2.sort_values(by=['INST_CARACTER_ACADEMICO'], ascending=True)
print(h2)

In [ ]:
#test de kruskal

l = data.INST_CARACTER_ACADEMICO.value_counts().index
pvalues = []
s_ = []

for k,m in enumerate(l):
  s = data.loc[data["INST_CARACTER_ACADEMICO"]== m,"PUNT_GLOBAL"]
  s_.append(s)
  test = stats.normaltest(s)
  pvalues.append(test.pvalue)

kw_test = stats.kruskal(*s_)

In [ ]:
#resultado importante del test (medianas por ser no parametrico)
kw_test

In [ ]:
pvalues

In [ ]:
# Supongamos que 's_' es tu lista de series, donde cada serie corresponde a un grupo diferente.
# El número total de comparaciones es n*(n-1)/2, donde n es el número de grupos.

# Calcula el número total de comparaciones
num_comparisons = len(s_) * (len(s_) - 1) / 2

# Nivel de significancia ajustado
alpha_adjusted = 0.05 / num_comparisons

# Lista para almacenar los resultados
post_hoc_results = []

# Itera sobre cada combinación única de pares de grupos
for group1, group2 in itertools.combinations(range(len(s_)), 2):
    stat, pvalue = mannwhitneyu(s_[group1], s_[group2], alternative='two-sided')
    # Añade los resultados a la lista
    post_hoc_results.append({
        'Grupo1': group1,
        'Grupo2': group2,
        'U-Statistic': stat,
        'P-Value': pvalue,
        'Significativo': pvalue < alpha_adjusted
    })

# Convierte los resultados a un DataFrame para una mejor visualización
post_hoc_df = pd.DataFrame(post_hoc_results)

# Muestra los resultados
print(post_hoc_df)

In [ ]:
icaracter = {
    "TÉCNICA PROFESIONAL" : "TÉCNICA O TECNOLÓGICA",
    "INSTITUCIÓN TECNOLÓGICA": "TÉCNICA O TECNOLÓGICA"
} # Agruapación de valores

In [ ]:
data['INST_CARACTER_ACADEMICOB']= data['INST_CARACTER_ACADEMICO'].replace(icaracter) # reemplazo de valores de acuerdo a la categoría establecida

In [ ]:
h3B = data.groupby('INST_CARACTER_ACADEMICOB').mean()
h3B = h3B.sort_values(by = ['PUNT_GLOBAL'], ascending = True)
h3B

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5), dpi = 100)
mat_punt = sns.violinplot(x = data['INST_CARACTER_ACADEMICOB'], y = data["PUNT_GLOBAL"], ax = ax, order=h3B.index)
labs = ax.set(xlabel='Condición', ylabel='Puntaje')
plt.xticks(rotation = -30, ha = "left")

###tablas dinamicas

In [ ]:
# Realizas la agrupación por las categorías deseadas
grouped = data.groupby(['ESTU_VALORMATRICULAUNIVERSIDADB', 'ESTU_METODO_PRGM', 'INST_CARACTER_ACADEMICOB'])

# Calculas el promedio de 'PUNT_GLOBAL'
mean_scores = grouped['PUNT_GLOBAL'].mean().reset_index(name='MEAN_PUNT_GLOBAL')

# Calculas el conteo para cada grupo usando la misma agrupación
counts = grouped['PUNT_GLOBAL'].count().reset_index(name='COUNT')

# Combina los resultados del promedio con los conteos
result = pd.merge(mean_scores, counts, on=['ESTU_METODO_PRGM', 'INST_CARACTER_ACADEMICOB', 'ESTU_VALORMATRICULAUNIVERSIDADB'])

result

In [ ]:
# Ordenas el DataFrame por el promedio de 'PUNT_GLOBAL' de forma descendente
result.sort_values(by='MEAN_PUNT_GLOBAL', ascending=False).reset_index(drop=True)

# arbol de decision


## split data

In [ ]:
select_cols = ['ESTU_VALORMATRICULAUNIVERSIDADB','ESTU_METODO_PRGM','INST_CARACTER_ACADEMICOB','PUNT_GLOBAL']

In [ ]:
df = data[select_cols]

In [ ]:
df = df.dropna()

In [ ]:
df.describe()

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train.PUNT_GLOBAL.values
y_test = df_test.PUNT_GLOBAL.values

In [ ]:
del df_train['PUNT_GLOBAL']
del df_test['PUNT_GLOBAL']

In [ ]:
train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [ ]:
test_dicts = df_test.to_dict(orient='records')
X_test = dv.transform(test_dicts)

##arbol de regresión

In [ ]:
dt = DecisionTreeRegressor(max_depth=4)
dt.fit(X_train, y_train)

In [ ]:
feature_names = dv.get_feature_names_out()
print(feature_names)

In [ ]:
# Nuevo array con los nombres propuestos en orden
new_feature_names = [
    'Distancia',
    'DistVirDtual',
    'Presencial',
    'Valor_>7M',
    'Valor_1-2.5M',
    'Valor_2.5-4M',
    'Valor_4-5.5M',
    'Valor_5.5-7M',
    'Valor_<1M',
    'Inst_Universitaria',
    'Inst_Tecnica',
    'Universidad',
]

In [ ]:
print(export_text(dt, feature_names=new_feature_names))

In [ ]:
plt.figure(figsize=(17,6))
plot_tree(dt, feature_names=new_feature_names, filled=True, rounded = True, fontsize = 8)
plt.show()

In [ ]:
y_pred = dt.predict(X_test)

In [ ]:
ms = mean_squared_error(y_test, y_pred)

In [ ]:
RMSE = np.sqrt(ms)
RMSE

In [ ]:
importancesdt = dt.feature_importances_


In [ ]:
dt_importances = pd.Series(importancesdt, index=new_feature_names)

fig, ax = plt.subplots(1,1, figsize=(10,6), dpi = 100)
dt_importances.plot.bar( ax=ax)
ax.set_xticklabels(new_feature_names)
ax.set_title("Relevancia por variable mediante decrecimiento de la impureza media")
ax.set_ylabel("Decrecimiento de la impureza media")
fig.tight_layout()

In [ ]:
# Creamos la tabla de importancias
importance_table = pd.DataFrame({
    'Feature': dt_importances.index,
    'Importance': dt_importances.values
})

# Ordenamos la tabla por importancia de manera descendente
importance_table = importance_table.sort_values('Importance', ascending=False)

# Suponiendo que 'importance_table' es tu DataFrame con la tabla de importancia
importance_table['Importance'] = importance_table['Importance'].round(4)

# Mostramos la tabla
print(importance_table)

#Posibles inconsistencias

In [ ]:
data2 = df[(df['ESTU_VALORMATRICULAUNIVERSIDADB']=='Más de 7 millones') & (df['ESTU_METODO_PRGM'] != 'PRESENCIAL')]
#data = data2[]
data2

In [ ]:
data2 = df[(df['ESTU_VALORMATRICULAUNIVERSIDADB']=='Más de 7 millones') & (df['INST_CARACTER_ACADEMICOB'] != 'UNIVERSIDAD')]
#data = data2[]
data2

In [ ]:
data2 = data[(data['ESTU_VALORMATRICULAUNIVERSIDAD']=='Más de 7 millones') & (data['INST_CARACTER_ACADEMICO'] == 'TÉCNICA PROFESIONAL')]
#data = data2[]
data2

In [ ]:
    #"TÉCNICA PROFESIONAL" : "TÉCNICA O TECNOLÓGICA",
    #"INSTITUCIÓN TECNOLÓGICA": "TÉCNICA O TECNOLÓGICA"

data2 = data[(data['ESTU_VALORMATRICULAUNIVERSIDADB']=='entre 5.5 y 7 millones') & (data['INST_CARACTER_ACADEMICO'] == 'TÉCNICA PROFESIONAL')]
#data = data2[]
data2

In [ ]:
import os

# Busca el cuaderno en el directorio actual
nombre_cuaderno = "SABER PRO_RESULTADOS.ipynb"
ruta_cuaderno = nombre_cuaderno

if os.path.exists(ruta_cuaderno):
    print(f"El cuaderno {nombre_cuaderno} se encuentra en la ruta: {ruta_cuaderno}")
else:
    print(f"El cuaderno {nombre_cuaderno} no se encuentra en el directorio actual.")

# Intenta convertir el cuaderno solo si se encuentra
if os.path.exists(ruta_cuaderno):
    # Ejecuta nbconvert para convertir el cuaderno a PDF
    !jupyter nbconvert "$ruta_cuaderno" --to pdf --output-dir="/content/downloads"

    # Muestra un enlace para descargar el PDF
    ruta_pdf = f"/content/downloads/{nombre_cuaderno.replace('.ipynb', '.pdf')}"
    from IPython.display import FileLink
    FileLink(ruta_pdf)
else:
    print("No se puede convertir el cuaderno porque no se encuentra en el directorio actual.")
